In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tensorflow import keras
import seaborn as sns
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Input, Dense, Conv2D, Activation, MaxPool2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization, Flatten, Reshape, Conv2DTranspose, LeakyReLU
from tensorflow.keras.models import Model
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, RMSprop
from sklearn.utils import compute_class_weight
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

np.random.seed(72)
tf.random.set_seed(72)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
/job:localhost/replica:0/task:0/device:GPU:1 -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5



In [2]:
def evaluate_(model, generator_test):
    model.evaluate(generator_test)
    
    y_pred = model.predict(generator_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = generator_test.classes
    class_labels = list(generator_test.class_indices.keys())

    print(classification_report(y_true, y_pred_classes))
    cm = confusion_matrix(y_true, y_pred_classes)
    
    # Plotting the confusion matrix
    plt.figure(figsize=(8, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
    plt.show()

In [3]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns

def evaluate_(model, generator_test):
    model.evaluate(generator_test)
    
    y_pred = model.predict(generator_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = generator_test.classes
    class_labels = list(generator_test.class_indices.keys())

    print(classification_report(y_true, y_pred_classes))
    cm = confusion_matrix(y_true, y_pred_classes)
    
    # Plotting the confusion matrix
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
    plt.title('Confusion Matrix')
    
    # ROC curve
    plt.subplot(1, 2, 2)
    fpr = dict()
    tpr = dict()
    roc_auc = dict()

    for i in range(len(class_labels)):
        fpr[i], tpr[i], _ = roc_curve(y_true == i, y_pred[:, i])
        roc_auc[i] = auc(fpr[i], tpr[i])

    for i in range(len(class_labels)):
        plt.plot(fpr[i], tpr[i], label=f'{class_labels[i]} (AUC = {roc_auc[i]:.2f})')

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend(loc="lower right")

    plt.tight_layout()
    plt.show()

# Call the function with your model and test generator


In [4]:
train_dir = r"/kaggle/input/mango-leaf/mango-prepo/train"
test_dir = r"/kaggle/input/mango-leaf/mango-prepo/test"
datagen_train = ImageDataGenerator(rescale=1./255,
                                  width_shift_range=0.1,
                                  height_shift_range=0.1,
                                  horizontal_flip=True,
                                  vertical_flip=False)


datagen_test = ImageDataGenerator(rescale=1./255)


batch_size = 16
generator_train = datagen_train.flow_from_directory(directory=train_dir,
                                                    target_size=(112, 112),
                                                    batch_size=batch_size,
                                                    shuffle=True)

generator_test = datagen_test.flow_from_directory(directory=test_dir,
                                                  target_size=(112, 112),
                                                  batch_size=batch_size,
                                                  shuffle=False)
# Calculate class weights
labels = generator_train.classes
class_weights = compute_class_weight(class_weight="balanced", classes=np.unique(labels), y=labels)
class_weights = dict(zip(np.unique(labels), class_weights))
print(class_weights)

Found 13307 images belonging to 26 classes.
Found 1684 images belonging to 26 classes.
{0: 1.0640492563569486, 1: 1.0662660256410257, 2: 1.0662660256410257, 3: 1.0402595372107568, 4: 1.2273565762774397, 5: 1.0381494772975504, 6: 1.053102247546692, 7: 1.2795192307692307, 8: 1.0487862547288778, 9: 1.053102247546692, 10: 0.6486789509603198, 11: 1.0662660256410257, 12: 1.4623076923076923, 13: 0.8809082483781279, 14: 0.7571119708693673, 15: 1.0466414975617429, 16: 1.7527660695468914, 17: 0.5415954415954416, 18: 1.0662660256410257, 19: 0.9042538733351454, 20: 0.789826685660019, 21: 1.5462468045549616, 22: 1.0256667180514876, 23: 1.8085077466702908, 24: 1.0445054945054946, 25: 0.7259683578832515}


In [5]:
!pip install keras_cv_attention_models 

  Obtaining dependency information for keras_cv_attention_models from https://files.pythonhosted.org/packages/67/b4/a581ae34f6a37b021e32d1d874b8b7df4664cbb8a71659886193caf341e0/keras_cv_attention_models-1.3.22-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.8/183.8 kB 5.5 MB/s eta 0:00:00
  Obtaining dependency information for ftfy from https://files.pythonhosted.org/packages/91/f8/dfa32d06cfcbdb76bc46e0f5d69c537de33f4cedb1a15cd4746ab45a6a26/ftfy-6.1.3-py3-none-any.whl.metadata
  Obtaining dependency information for wcwidth<0.3.0,>=0.2.12 from https://files.pythonhosted.org/packages/31/b1/a59de0ad3aabb17523a39804f4c6df3ae87ead053a4e25362ae03d73d03a/wcwidth-0.2.12-py2.py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 773.2/773.2 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 3.0 MB/s eta 0:00:00
  Attempting uninstall: wcwidth
    Found existing installation: wcwidth 0.2.6
    Uninstalling wcwidth-0.2.6:
 

In [ ]:
np.random.seed(72)
tf.random.set_seed(72)
sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(log_device_placement=True))


In [8]:
from keras_cv_attention_models import wave_mlp

In [ ]:
pip install keras-self-attention


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Layer, Add, Input, Conv2D, BatchNormalization, Activation, GlobalAveragePooling2D, Reshape, Concatenate, Lambda, Permute
from tensorflow.keras import Model

class SelfAttention(Layer):
    def __init__(self, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W_q = self.add_weight(name='W_q', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        self.W_k = self.add_weight(name='W_k', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        self.W_v = self.add_weight(name='W_v', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        super(SelfAttention, self).build(input_shape)

    def call(self, x):
        q = x @ self.W_q
        k = x @ self.W_k
        v = x @ self.W_v

        attn_score = tf.matmul(q, k, transpose_b=True)
        attn_score = tf.nn.softmax(attn_score, axis=-1)

        output = attn_score @ v
        return output

    def compute_output_shape(self, input_shape):
        return input_shape

class PositionalEncoding(Layer):
    def __init__(self, input_shape, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.input_shape_ = input_shape

    def build(self, input_shape):
        self.positional_encoding = self.add_weight(name='positional_encoding',
                                                   shape=(1, *self.input_shape_[1:], 1),
                                                   initializer='uniform',
                                                   trainable=True)
        super(PositionalEncoding, self).build(input_shape)

    def call(self, x):
        return x + self.positional_encoding

    def compute_output_shape(self, input_shape):
        return input_shape

def shufflenet_block(x, groups):
    _, _, _, c = x.get_shape().as_list()

    # Channel split
    x = Reshape((-1, groups))(x)
    x = Permute((2, 1))(x)
    x = Reshape((-1, c))(x)

    # Channel shuffle
    x = Lambda(lambda z: tf.keras.backend.batch_flatten(tf.keras.backend.permute_dimensions(z, (0, 2, 1))))(x)

    # Pointwise group convolution
    x = Reshape((1, 1, c))(x)
    x = Conv2D(c, kernel_size=(1, 1), groups=groups, use_bias=False)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)

    # Global Average Pooling
    x = GlobalAveragePooling2D()(x)

    # Reshape for concatenation
    x = Reshape((1, 1, c))(x)

    # Concatenate with the original input
    x = Concatenate(axis=-1)([x, x])

    return x

def custom_head_with_shufflenet(input_tensor, num_classes):
    x = Flatten()(input_tensor)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Add ShuffleNet block
    shufflenet_output = shufflenet_block(tf.keras.layers.Reshape((1, 1, 256))(x), groups=4)
    x = Add()([input_tensor, shufflenet_output])

    # Add self-attention mechanism
    self_attention = SelfAttention()(x)
    x = Add()([x, self_attention])

    # Add 2D positional encoding
    position_encoding = PositionalEncoding(input_shape=(1, 1, 256))(x)
    x = Add()([x, position_encoding])

    x = Dense(num_classes, activation='softmax')(x)
    return x

def modify_wave_mlp(input_shape, num_classes):
    mm_headless = wave_mlp.WaveMLP_T(input_shape=input_shape, pretrained="imagenet")
    head_output = custom_head_with_shufflenet(mm_headless.output, num_classes)
    custom_model = Model(inputs=mm_headless.input, outputs=head_output)
    return custom_model

# Example usage:
input_shape = (112, 112, 3)
num_classes = 26  # Adjust based on your task

custom_model = modify_wave_mlp(input_shape, num_classes)
custom_model.summary()


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Layer, Add
from tensorflow.keras import Input, Model
# Import your wave_mlp module

class SelfAttention(Layer):
    def __init__(self, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W_q = self.add_weight(name='W_q', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        self.W_k = self.add_weight(name='W_k', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        self.W_v = self.add_weight(name='W_v', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        super(SelfAttention, self).build(input_shape)

    def call(self, x):
        q = x @ self.W_q
        k = x @ self.W_k
        v = x @ self.W_v

        attn_score = tf.matmul(q, k, transpose_b=True)
        attn_score = tf.nn.softmax(attn_score, axis=-1)

        output = attn_score @ v
        return output

    def compute_output_shape(self, input_shape):
        return input_shape

class PositionalEncoding(Layer):
    def __init__(self, input_shape, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.input_shape_ = input_shape

    def build(self, input_shape):
        self.positional_encoding = self.add_weight(name='positional_encoding',
                                                   shape=(1, *self.input_shape_[1:], 1),
                                                   initializer='uniform',
                                                   trainable=True)
        super(PositionalEncoding, self).build(input_shape)

    def call(self, x):
        return x + self.positional_encoding

    def compute_output_shape(self, input_shape):
        return input_shape

class ResidualBlock(Layer):
    def __init__(self, **kwargs):
        super(ResidualBlock, self).__init__(**kwargs)
        self.conv1 = tf.keras.layers.Conv2D(filters=None, kernel_size=(3, 3), padding='same', activation='relu')
        self.conv2 = tf.keras.layers.Conv2D(filters=None, kernel_size=(3, 3), padding='same', activation='relu')
        self.add = Add()

    def build(self, input_shape):
        # Infer the number of filters dynamically based on the input shape
        num_filters = input_shape[-1]
        self.conv1.filters = num_filters
        self.conv2.filters = num_filters
        super(ResidualBlock, self).build(input_shape)

    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.conv2(x)
        return self.add([inputs, x])



def custom_head_with_stages(input_tensor, num_classes):
    # Stage 1: Global Self-Attention
    x = SelfAttention()(input_tensor)

    # Stage 2: Global Self-Attention
    x = SelfAttention()(x)

    # Stage 3: Global Self-Attention
    x = SelfAttention()(x)

    # Stage 4: Residual Blocks
    x = ResidualBlock(256, (3, 3))(x)
    x = ResidualBlock(256, (3, 3))(x)

    # Flatten and add your custom dense layers
    x = Flatten()(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Add self-attention mechanism
    self_attention = SelfAttention()(x)
    x = Add()([x, self_attention])

    # Add 2D positional encoding
    position_encoding = PositionalEncoding(input_shape=(1, 1, 256))(x)
    x = Add()([x, position_encoding])

    # Final Dense layer
    x = Dense(num_classes, activation='softmax')(x)

    return x

def modify_wave_mlp_with_stages(input_shape, num_classes):
    # Load the WaveMLP model without the top layers (head)
    mm_headless = wave_mlp.WaveMLP_T(input_shape=input_shape, pretrained="imagenet")

    # Add your custom head with stages
    head_output = custom_head_with_stages(mm_headless.output, num_classes)

    # Create the custom model by combining the base model and the custom head
    custom_model = Model(inputs=mm_headless.input, outputs=head_output)

    return custom_model

# Example usage:
input_shape = (112, 112, 3)
num_classes = 26  # Adjust based on your task

custom_model_with_stages = modify_wave_mlp_with_stages(input_shape, num_classes)
custom_model_with_stages.summary()


In [9]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Layer, Add, Conv2D, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras import Input, Model
# Import your wave_mlp module

class SelfAttention(Layer):
    def __init__(self, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W_q = self.add_weight(name='W_q', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        self.W_k = self.add_weight(name='W_k', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        self.W_v = self.add_weight(name='W_v', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        super(SelfAttention, self).build(input_shape)

    def call(self, x):
        q = x @ self.W_q
        k = x @ self.W_k
        v = x @ self.W_v

        attn_score = tf.matmul(q, k, transpose_b=True)
        attn_score = tf.nn.softmax(attn_score, axis=-1)

        output = attn_score @ v
        return output

    def compute_output_shape(self, input_shape):
        return input_shape

class PositionalEncoding(Layer):
    def __init__(self, input_shape, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.input_shape_ = input_shape

    def build(self, input_shape):
        self.positional_encoding = self.add_weight(name='positional_encoding',
                                                   shape=(1, *self.input_shape_[1:], 1),
                                                   initializer='uniform',
                                                   trainable=True)
        super(PositionalEncoding, self).build(input_shape)

    def call(self, x):
        return x + self.positional_encoding

    def compute_output_shape(self, input_shape):
        return input_shape

def custom_head(input_tensor, num_classes):
    x = Flatten()(input_tensor)
    x = Dense(256, activation='relu')(x)  # Add your own dense layers
    x = Dropout(0.5)(x)  # Add dropout for regularization

    # Add self-attention mechanism
    self_attention = SelfAttention()(x)
    x = Add()([x, self_attention])

    # Add 2D positional encoding
    position_encoding = PositionalEncoding(input_shape=(1, 1, 256))(x)  # Adjust the input_shape
    x = Add()([x, position_encoding])

    x = Dense(num_classes, activation='softmax')(x)
    return x

def stage_block(x, filters, num_blocks, attention=True, hdc=False):
    # Check if the input is flattened
    if len(x.shape) == 2:
        # Reshape to 4D tensor
        x = tf.reshape(x, [-1, int(x.shape[1] ** 0.5), int(x.shape[1] ** 0.5), 1])

    for _ in range(num_blocks):
        # Residual block
        residual = x

        # Global self-attention
        if attention:
            x = SelfAttention()(x)

        # Convolutional layer with Hybrid Dilated Convolution (HDC)
        if hdc:
            x = Conv2D(filters, kernel_size=(3, 3), dilation_rate=(2, 2), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
            x = Conv2D(filters, kernel_size=(3, 3), dilation_rate=(3, 3), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)
        else:
            x = Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
            x = BatchNormalization()(x)
            x = Activation('relu')(x)

        # Residual connection (adjusting the residual size)
        residual = Conv2D(filters, kernel_size=(1, 1), padding='same')(residual)
        x = Add()([x, residual])

    # MaxPooling for downsampling
    x = MaxPooling2D(pool_size=(2, 2))(x)

    return x

def modify_wave_mlp(input_shape, num_classes):
    # Load the WaveMLP model without the top layers (head)
    mm_headless = wave_mlp.WaveMLP_T(input_shape=input_shape, pretrained="imagenet")

    # Stage 1 with HDC
    x = stage_block(mm_headless.output, filters=64, num_blocks=3, hdc=True)

    # Stage 2 with HDC
    x = stage_block(x, filters=128, num_blocks=3, hdc=True)

    # Stage 3 with HDC
    x = stage_block(x, filters=256, num_blocks=3, hdc=True)

    # Stage 4 (Residual blocks)
    x = stage_block(x, filters=512, num_blocks=3, attention=False, hdc=False)

    # Custom head
    head_output = custom_head(x, num_classes)

    # Create the custom model by combining the base model and the custom head
    custom_model = Model(inputs=mm_headless.input, outputs=head_output)

    return custom_model

# Example usage:
input_shape = (112, 112, 3)
num_classes = 26  # Adjust based on your task

custom_model = modify_wave_mlp(input_shape, num_classes)
custom_model.summary()


69906376/69906376 [==============================] - 1s 0us/step
>>>> Load pretrained from: /root/.keras/models/wavemlp_t_imagenet.h5
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 112, 112, 3)]        0         []                            
                                                                                                  
 stem_pad (ZeroPadding2D)    (None, 116, 116, 3)          0         ['input_1[0][0]']             
                                                                                                  
 stem_conv (Conv2D)          (None, 28, 28, 64)           9472      ['stem_pad[0][0]']            
                                                                                                  
 stem_bn (BatchNormalizatio  (None, 28, 28, 64)           2

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Layer, Add, Conv2D, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras import Input, Model
# Import your wave_mlp module

class SelfAttention(Layer):
    def __init__(self, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W_q = self.add_weight(name='W_q', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        self.W_k = self.add_weight(name='W_k', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        self.W_v = self.add_weight(name='W_v', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        super(SelfAttention, self).build(input_shape)

    def call(self, x):
        q = x @ self.W_q
        k = x @ self.W_k
        v = x @ self.W_v

        attn_score = tf.matmul(q, k, transpose_b=True)
        attn_score = tf.nn.softmax(attn_score, axis=-1)

        output = attn_score @ v
        return output

    def compute_output_shape(self, input_shape):
        return input_shape

class PositionalEncoding(Layer):
    def __init__(self, input_shape, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.input_shape_ = input_shape

    def build(self, input_shape):
        self.positional_encoding = self.add_weight(name='positional_encoding',
                                                   shape=(1, *self.input_shape_[1:], 1),
                                                   initializer='uniform',
                                                   trainable=True)
        super(PositionalEncoding, self).build(input_shape)

    def call(self, x):
        return x + self.positional_encoding

    def compute_output_shape(self, input_shape):
        return input_shape

def custom_head(input_tensor, num_classes):
    x = Flatten()(input_tensor)
    x = Dense(256, activation='relu')(x)  # Add your own dense layers
    x = Dropout(0.5)(x)  # Add dropout for regularization

    # Add self-attention mechanism
    self_attention = SelfAttention()(x)
    x = Add()([x, self_attention])

    # Add 2D positional encoding
    position_encoding = PositionalEncoding(input_shape=(1, 1, 256))(x)  # Adjust the input_shape
    x = Add()([x, position_encoding])

    x = Dense(num_classes, activation='softmax')(x)
    return x

def stage_block(x, filters, num_blocks, attention=True):
    # Check if the input is flattened
    if len(x.shape) == 2:
        # Reshape to 4D tensor
        x = tf.reshape(x, [-1, int(x.shape[1] ** 0.5), int(x.shape[1] ** 0.5), 1])

    for _ in range(num_blocks):
        # Residual block
        residual = x

        # Global self-attention
        if attention:
            x = SelfAttention()(x)

        # Convolutional layer
        x = Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # Residual connection (adjusting the residual size)
        residual = Conv2D(filters, kernel_size=(1, 1), padding='same')(residual)
        x = Add()([x, residual])

    # MaxPooling for downsampling
    x = MaxPooling2D(pool_size=(2, 2))(x)

    return x



def modify_wave_mlp(input_shape, num_classes):
    # Load the WaveMLP model without the top layers (head)
    mm_headless = wave_mlp.WaveMLP_T(input_shape=input_shape, pretrained="imagenet")

    # Stage 1
    x = stage_block(mm_headless.output, filters=64, num_blocks=3)

    # Stage 2
    x = stage_block(x, filters=128, num_blocks=3)

    # Stage 3
    x = stage_block(x, filters=256, num_blocks=3)

    # Stage 4 (Residual blocks)
    x = stage_block(x, filters=512, num_blocks=3, attention=False)

    # Custom head
    head_output = custom_head(x, num_classes)

    # Create the custom model by combining the base model and the custom head
    custom_model = Model(inputs=mm_headless.input, outputs=head_output)

    return custom_model

# Example usage:
input_shape = (112, 112, 3)
num_classes = 26  # Adjust based on your task

custom_model = modify_wave_mlp(input_shape, num_classes)
custom_model.summary()


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Layer, Add, Conv2D, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras import Input, Model

# Define the MultiheadSelfAttentionBlock as a custom layer
class MultiheadSelfAttentionBlock(Layer):
    def __init__(self, embedding_dim=256, num_heads=4, attn_dropout=0.1):
        super(MultiheadSelfAttentionBlock, self).__init__()

        # Create the Layer Normalization (LN)
        self.layer_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        # Create the Multi-Head Attention (MSA) layer
        self.multihead_attn = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embedding_dim // num_heads,
            dropout=attn_dropout
        )

    def call(self, x):
        # Reshape to (batch_size, sequence_length, features)
        original_shape = tf.shape(x)
        if len(original_shape) == 2:
            x = tf.expand_dims(x, axis=1)
        x = self.layer_norm(x)
        attn_output = self.multihead_attn(x, x, x)

        # Reshape back to the original shape
        attn_output = tf.reshape(attn_output, original_shape)

        return attn_output

def stage_block(x, filters, num_blocks, attention=True):
    # Check if the input is flattened
    if len(x.shape) == 2:
        # Reshape to 4D tensor
        x = tf.reshape(x, [-1, int(x.shape[1] ** 0.5), int(x.shape[1] ** 0.5), 1])

    for _ in range(num_blocks):
        # Residual block
        residual = x

        # Global self-attention
        if attention:
            x = MultiheadSelfAttentionBlock()(x)

        # Convolutional layer
        x = Conv2D(filters, kernel_size=(3, 3), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        # Residual connection (adjusting the residual size)
        residual = Conv2D(filters, kernel_size=(1, 1), padding='same')(residual)
        x = Add()([x, residual])

    # MaxPooling for downsampling
    x = MaxPooling2D(pool_size=(2, 2))(x)

    return x


def custom_head(input_tensor, num_classes):
    x = Flatten()(input_tensor)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Add self-attention mechanism
    self_attention = MultiheadSelfAttentionBlock()(x)
    x = Add()([x, self_attention])

    # Add 2D positional encoding
    position_encoding = PositionalEncoding(input_shape=(1, 1, 256))(x)
    x = Add()([x, position_encoding])

    # Add Multihead Self-Attention Block
    multihead_attention_block = MultiheadSelfAttentionBlock(embedding_dim=256, num_heads=4, attn_dropout=0.1)
    x = multihead_attention_block(x)

    x = Dense(num_classes, activation='softmax')(x)
    return x

def modify_wave_mlp(input_shape, num_classes):
    # Load the WaveMLP model without the top layers (head)
    mm_headless = wave_mlp.WaveMLP_T(input_shape=input_shape, pretrained="imagenet")

    # Stage 1
    x = stage_block(mm_headless.output, filters=64, num_blocks=3)

    # Stage 2
    x = stage_block(x, filters=128, num_blocks=3)

    # Stage 3
    x = stage_block(x, filters=256, num_blocks=3)

    # Stage 4 (Residual blocks)
    x = stage_block(x, filters=512, num_blocks=3, attention=False)

    # Custom head
    head_output = custom_head(x, num_classes)

    # Create the custom model by combining the base model and the custom head
    custom_model = Model(inputs=mm_headless.input, outputs=head_output)

    return custom_model

# Example usage:
input_shape = (112, 112, 3)
num_classes = 26  # Adjust based on your task

custom_model = modify_wave_mlp(input_shape, num_classes)
custom_model.summary()


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Layer, Add, Conv2D, BatchNormalization, Activation, MaxPooling2D
from tensorflow.keras import Input, Model

# Define the MultiheadSelfAttentionBlock as a custom layer
class MultiheadSelfAttentionBlock(Layer):
    def __init__(self, embedding_dim=256, num_heads=4, attn_dropout=0.1):
        super(MultiheadSelfAttentionBlock, self).__init__()

        # Create the Layer Normalization (LN)
        self.layer_norm = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        # Create the Multi-Head Attention (MSA) layer
        self.multihead_attn = tf.keras.layers.MultiHeadAttention(
            num_heads=num_heads,
            key_dim=embedding_dim // num_heads,
            dropout=attn_dropout
        )

    def call(self, x):
        x = self.layer_norm(x)
        attn_output = self.multihead_attn(x, x, x)
        return attn_output

# Your existing code...

def custom_head(input_tensor, num_classes):
    x = Flatten()(input_tensor)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Add self-attention mechanism
    self_attention = SelfAttention()(x)
    x = Add()([x, self_attention])

    # Add 2D positional encoding
    position_encoding = PositionalEncoding(input_shape=(1, 1, 256))(x)
    x = Add()([x, position_encoding])

    # Add Multihead Self-Attention Block
    multihead_attention_block = MultiheadSelfAttentionBlock(embedding_dim=256, num_heads=4, attn_dropout=0.1)
    x = multihead_attention_block(x)

    x = Dense(num_classes, activation='softmax')(x)
    return x

def modify_wave_mlp(input_shape, num_classes):
    # Load the WaveMLP model without the top layers (head)
    mm_headless = wave_mlp.WaveMLP_T(input_shape=input_shape, pretrained="imagenet")

    # Stage 1
    x = stage_block(mm_headless.output, filters=64, num_blocks=3)

    # Stage 2
    x = stage_block(x, filters=128, num_blocks=3)

    # Stage 3
    x = stage_block(x, filters=256, num_blocks=3)

    # Stage 4 (Residual blocks)
    x = stage_block(x, filters=512, num_blocks=3, attention=False)

    # Custom head
    head_output = custom_head(x, num_classes)

    # Create the custom model by combining the base model and the custom head
    custom_model = Model(inputs=mm_headless.input, outputs=head_output)

    return custom_model

# Example usage:
input_shape = (112, 112, 3)
num_classes = 26  # Adjust based on your task

custom_model = modify_wave_mlp(input_shape, num_classes)
custom_model.summary()


In [ ]:
from keras_self_attention import MultiHeadAttention

def custom_head(input_tensor, num_classes):
    x = Flatten()(input_tensor)
    
    # Add multi-headed self-attention layer
    attention = MultiHeadAttention(head_size=128, num_heads=4)(input_tensor)
    x = Concatenate()([input_tensor, attention])  # Concatenate attention output with input
    
    x = Dense(256, activation='relu')(x)  # Add your own dense layers
    x = Dropout(0.5)(x)  # Add dropout for regularization
    x = Dense(num_classes, activation='softmax')(x)
    return x

def modify_wave_mlp(input_shape, num_classes):
    # Load the WaveMLP model without the top layers (head)
    mm_headless = wave_mlp.WaveMLP_T(input_shape=input_shape, pretrained="imagenet")

    # Add your custom head
    head_output = custom_head(mm_headless.output, num_classes)

    # Create the custom model by combining the base model and the custom head
    custom_model = Model(inputs=mm_headless.input, outputs=head_output)

    return custom_model

# Example usage:
input_shape = (112, 112, 3)
num_classes = 26  # Adjust based on your task

custom_model = modify_wave_mlp(input_shape, num_classes)
custom_model.summary()


In [ ]:
from tensorflow.keras.layers import Flatten, Dense, Dropout, LayerNormalization, MultiHeadAttention, Add, Input, Embedding
from tensorflow.keras.models import Model

def custom_head(input_tensor, num_classes):
    x = Flatten()(input_tensor)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)

    # Self-Attention Mechanism
    att_input = Input(shape=(x.shape[1],))
    att_output = MultiHeadAttention(num_heads=8, key_dim=16)([att_input, att_input])
    att_output = Add()([att_input, att_output])
    att_output = LayerNormalization(epsilon=1e-6)(att_output)

    # Positional Encoding
    pos_input = Input(shape=(x.shape[1],))
    pos_output = Embedding(input_dim=x.shape[1], output_dim=256)(pos_input)
    pos_output = Add()([att_output, pos_output])
    pos_output = LayerNormalization(epsilon=1e-6)(pos_output)

    # Combine attention and positional encoding
    x = Add()([att_output, pos_output])

    # Fully connected layer for classification
    x = Dense(num_classes, activation='softmax')(x)
    return x

def modify_wave_mlp(input_shape, num_classes):
    # Load the WaveMLP model without the top layers (head)
    mm_headless = wave_mlp.WaveMLP_T(input_shape=input_shape, pretrained="imagenet")

    # Add your custom head
    head_output = custom_head(mm_headless.output, num_classes)

    # Create the custom model by combining the base model and the custom head
    custom_model = Model(inputs=mm_headless.input, outputs=head_output)

    return custom_model

# Example usage:
input_shape = (112, 112, 3)
num_classes = 26  # Adjust based on your task

custom_model = modify_wave_mlp(input_shape, num_classes)
# custom_model.summary()


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Flatten, Dense, Dropout, Layer, Add
from tensorflow.keras import Input, Model
 # Import your wave_mlp module

class SelfAttention(Layer):
    def __init__(self, **kwargs):
        super(SelfAttention, self).__init__(**kwargs)

    def build(self, input_shape):
        self.W_q = self.add_weight(name='W_q', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        self.W_k = self.add_weight(name='W_k', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        self.W_v = self.add_weight(name='W_v', shape=(input_shape[-1], input_shape[-1]),
                                   initializer='uniform', trainable=True)
        super(SelfAttention, self).build(input_shape)

    def call(self, x):
        q = x @ self.W_q
        k = x @ self.W_k
        v = x @ self.W_v

        attn_score = tf.matmul(q, k, transpose_b=True)
        attn_score = tf.nn.softmax(attn_score, axis=-1)

        output = attn_score @ v
        return output

    def compute_output_shape(self, input_shape):
        return input_shape

class PositionalEncoding(Layer):
    def __init__(self, input_shape, **kwargs):
        super(PositionalEncoding, self).__init__(**kwargs)
        self.input_shape_ = input_shape

    def build(self, input_shape):
        self.positional_encoding = self.add_weight(name='positional_encoding',
                                                   shape=(1, *self.input_shape_[1:], 1),
                                                   initializer='uniform',
                                                   trainable=True)
        super(PositionalEncoding, self).build(input_shape)

    def call(self, x):
        return x + self.positional_encoding

    def compute_output_shape(self, input_shape):
        return input_shape

def custom_head(input_tensor, num_classes):
    x = Flatten()(input_tensor)
    x = Dense(256, activation='relu')(x)  # Add your own dense layers
    x = Dropout(0.5)(x)  # Add dropout for regularization

    # Add self-attention mechanism
    self_attention = SelfAttention()(x)
    x = Add()([x, self_attention])

    # Add 2D positional encoding
    position_encoding = PositionalEncoding(input_shape=(1, 1, 256))(x)  # Adjust the input_shape
    x = Add()([x, position_encoding])

    x = Dense(num_classes, activation='softmax')(x)
    return x

def modify_wave_mlp(input_shape, num_classes):
    # Load the WaveMLP model without the top layers (head)
    mm_headless = wave_mlp.WaveMLP_T(input_shape=input_shape, pretrained="imagenet")

    # Add your custom head
    head_output = custom_head(mm_headless.output, num_classes)

    # Create the custom model by combining the base model and the custom head
    custom_model = Model(inputs=mm_headless.input, outputs=head_output)

    return custom_model

# Example usage:
input_shape = (112, 112, 3)
num_classes = 26  # Adjust based on your task

custom_model = modify_wave_mlp(input_shape, num_classes)
# custom_model.summary()


In [ ]:
def custom_head(input_tensor, num_classes):
    x = Flatten()(input_tensor)
    x = Dense(256, activation='relu')(x)  # Add your own dense layers
    x = Dropout(0.5)(x)  # Add dropout for regularization
    x = Dense(num_classes, activation='softmax')(x)
    return x



def modify_wave_mlp(input_shape, num_classes):
    # Load the WaveMLP model without the top layers (head)
    mm_headless = wave_mlp.WaveMLP_T(input_shape=input_shape, pretrained="imagenet")

    # Add your custom head
#     head_output = custom_head(mm_headless.output, num_classes)
    print("mm_headless.output shape:", mm_headless.output_shape)
    head_output = custom_head(mm_headless.output, num_classes)


    # Create the custom model by combining the base model and the custom head
    custom_model = Model(inputs=mm_headless.input, outputs=head_output)

    return custom_model

# Example usage:
input_shape = (112, 112, 3)
num_classes = 26  # Adjust based on your task

custom_model = modify_wave_mlp(input_shape, num_classes)
# custom_model.summary()

In [ ]:
# def custom_head(input_tensor, num_classes):
#     x = Flatten()(input_tensor)
#     x = Dense(512, activation='relu')(x)
#     x = Dropout(0.3)(x)
#     x = Dense(256, activation='relu')(x)
#     x = Dropout(0.3)(x)
#     x = Dense(num_classes, activation='softmax')(x)
#     return x

# def modify_wave_mlp(input_shape, num_classes):
#     # Load the WaveMLP model without the top layers (head)
#     mm_headless = wave_mlp.WaveMLP_T(input_shape=input_shape, pretrained="imagenet")

#     # Add your custom head
#     head_output = custom_head(mm_headless.output, num_classes)

#     # Create the custom model by combining the base model and the custom head
#     custom_model = Model(inputs=mm_headless.input, outputs=head_output)

#     # Fine-tune the last few layers of the base model
#     for layer in mm_headless.layers[:-5]:
#         layer.trainable = True

#     # Compile the model with a custom learning rate and metrics
#     custom_model.compile(optimizer=Adam(lr=1e-4), loss='categorical_crossentropy', metrics=['accuracy', 'precision', 'recall', 'f1'])

#     return custom_model

# # Example usage:
# input_shape = (112, 112, 3)
# num_classes = 26  # Adjust based on your task

# custom_model = modify_wave_mlp(input_shape, num_classes)
# custom_model.summary()

In [ ]:
mm_last_layer = custom_model .get_layer('avg_pool').output
#out = Dense(256, activation='relu', name='dense_1')(mm_last_layer)
#out = Dense(11, activation='softmax', name='prediction1')(out)
mm_custom = Model(custom_model .input, mm_last_layer)

In [ ]:
from tensorflow.keras import layers
inputs = keras.Input(shape=(112,112,3))
outputs = layers.average([mm_custom(inputs)])

avg_ensemble_model = keras.Model(inputs=inputs, outputs=outputs)
avg_ensemble_model.summary()

In [ ]:
print("mm_headless.output shape:", mm_headless.output_shape)
head_output = custom_head(mm_headless.output, num_classes)


In [ ]:
from keras_cv_attention_models import coatnet
mm = coatnet.CoAtNet0(input_shape=(112, 112, 3), pretrained="imagenet")

In [ ]:
from keras_cv_attention_models import coatnet
mm = coatnet.CoAtNet0(input_shape=(112, 112, 3), pretrained="imagenet")

In [ ]:
from keras_cv_attention_models import res_mlp
# mm = res_mlp.ResMLP12()

In [ ]:
mm = res_mlp.ResMLP12(input_shape=(112, 112, 3), pretrained="imagenet")

In [ ]:
from keras_cv_attention_models import wave_mlp
mm = wave_mlp.WaveMLP_T(input_shape=(112, 112, 3), pretrained="imagenet")

In [ ]:
from keras_cv_attention_models import mobilevit
mm = mobilevit.MobileViTBasePatch16(input_shape=(112, 112, 3))
mm2 = mobilevit.MobileViTBasePatch16(input_shape=(112, 112, 3))


In [ ]:
# mm.summary()

In [ ]:
from keras_cv_attention_models import swin_transformer_v2

In [ ]:
mm2 = swin_transformer_v2.SwinTransformerV2Tiny_window8(input_shape=(112, 112, 3))

In [ ]:
# mm2.summary()

In [ ]:
"""
transfer_layer = mm.get_layer('avg_pool')
conv_model = Model(inputs=mm.input, outputs=transfer_layer.output)
"""

In [ ]:
"""
#for layer in conv_model.layers:
#    layer.trainable = False
    
# Start a new Keras Sequential model.
new_model = Sequential()

# Add the convolutional part of the VGG16 model from above.
new_model.add(conv_model)


# Add the final layer for the actual classification.
new_model.add(Dense(2, activation='softmax'))
"""

In [ ]:
from keras_cv_attention_models import beit
mm2 = beit.BeitBasePatch16(input_shape=(112, 112, 3))

In [ ]:
mm_last_layer = mm.get_layer('avg_pool').output
#out = Dense(256, activation='relu', name='dense_1')(mm_last_layer)
#out = Dense(11, activation='softmax', name='prediction1')(out)
mm_custom = Model(mm.input, mm_last_layer)

In [ ]:
mm2_last_layer = mm2.get_layer('out_ln').output
#out2 = Dense(256, activation='relu', name='dense_1')(mm2_last_layer)
#out2 = Dense(11, activation='softmax', name='prediction1')(out2)
mm2_custom = Model(mm2.input, mm2_last_layer)

In [ ]:
# Define a custom linear attention layer
class LinearAttentionLayer(keras.layers.Layer):
    def __init__(self, units, **kwargs):
        super(LinearAttentionLayer, self).__init__(**kwargs)
        self.units = units

    def build(self, input_shape):
        self.W_q = self.add_weight(shape=(input_shape[-1], self.units), initializer='uniform', trainable=True)
        self.W_k = self.add_weight(shape=(input_shape[-1], self.units), initializer='uniform', trainable=True)
        self.W_v = self.add_weight(shape=(input_shape[-1], self.units), initializer='uniform', trainable=True)

    def call(self, inputs):
        Q = tf.matmul(inputs, self.W_q)
        K = tf.matmul(inputs, self.W_k)
        V = tf.matmul(inputs, self.W_v)

        attn_scores = tf.matmul(Q, K, transpose_b=True)
        attn_scores = tf.nn.softmax(attn_scores / tf.math.sqrt(tf.cast(self.units, tf.float32)), axis=-1)
        output = tf.matmul(attn_scores, V)

        return output

# ... Continue with your code ...

# Add the attention layer where needed in your model
num_classes = 2
avg_ensemble_model_last_layer = avg_ensemble_model.get_layer('average').output

# Add Linear Attention Layer here (for example, just before the output layer)
attention_output = LinearAttentionLayer(64)(avg_ensemble_model_last_layer)

output_layer = Dense(num_classes, activation='softmax', name='output_1')(attention_output)
final_model = Model(avg_ensemble_model.input, output_layer)

final_model.summary()

In [ ]:
from tensorflow.keras import layers
inputs = keras.Input(shape=(112,112,3))
outputs = layers.average([mm_custom(inputs), mm2_custom(inputs)])

avg_ensemble_model = keras.Model(inputs=inputs, outputs=outputs)
avg_ensemble_model.summary()

In [ ]:
from tensorflow.keras import layers
inputs = keras.Input(shape=(112,112,3))
outputs = layers.average([mm_custom(inputs)])

avg_ensemble_model = keras.Model(inputs=inputs, outputs=outputs)
avg_ensemble_model.summary()

In [ ]:
num_classes = 26
avg_ensemble_model_last_layer = avg_ensemble_model.get_layer('average_1').output
output_layer = Dense(num_classes, activation='softmax', name='output_1')(avg_ensemble_model_last_layer)
final_model = Model(avg_ensemble_model.input, output_layer)

final_model.summary()

In [ ]:
optimizer = Adam(lr=1e-5)
loss = 'categorical_crossentropy'
# metrics = ['categorical_accuracy']
metrics = ['accuracy', 'categorical_accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(), 
           tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), 
           tf.keras.metrics.FalseNegatives(), tfa.metrics.CohenKappa(num_classes = num_classes), 
           tfa.metrics.F1Score(num_classes = num_classes)]

final_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

In [ ]:
import os

# Delete the existing HDF5 file if it exists
if os.path.exists('Best_DenseNet201.h5'):
    os.remove('Best_DenseNet201.h5')

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
lr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.1,
    patience=9, mode="max", min_delta=0.0001, min_lr=0.00001, verbose=1)
checkpoint = ModelCheckpoint(filepath='Best_DenseNet201_v23.h5', save_best_only=True, monitor = 'val_accuracy', verbose=1)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=1, restore_best_weights=True)

callbacks = [lr, checkpoint, early_stopping]

In [ ]:
epochs = 30

steps_per_epoch = generator_train.n / batch_size
steps_test = generator_test.n / batch_size

history = final_model.fit_generator(generator=generator_train,
                                  epochs=epochs,
                                  steps_per_epoch=steps_per_epoch,
                                  validation_data=generator_test,
                                  validation_steps=steps_test,
                                   callbacks=callbacks, class_weight =class_weights)

In [ ]:
evaluate_(final_model, generator_test)

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['categorical_accuracy'])
plt.plot(history.history['val_categorical_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
!pip install --upgrade scipy scikit-image


In [ ]:
image_size = 128
batch_size = 8

train_dir = r"/kaggle/input/mango-leaf/mango-prepo/train"
val_dir = r"/kaggle/input/mango-leaf/mango-prepo/val"
test_dir = r"/kaggle/input/mango-leaf/mango-prepo/test"


datagen_train = ImageDataGenerator(rescale=1./255, width_shift_range=0.1, height_shift_range=0.1,
                                  horizontal_flip=True,  vertical_flip=False)
datagen_test = ImageDataGenerator(rescale=1./255)

train_generator = datagen_train.flow_from_directory(directory=train_dir, target_size=(image_size, image_size),
                                                    batch_size=batch_size, shuffle=True)
val_generator = datagen_test.flow_from_directory(directory=val_dir, target_size=(image_size, image_size),
                                                  batch_size=batch_size, shuffle=False)
test_generator = datagen_test.flow_from_directory(directory=test_dir, target_size=(image_size, image_size),
                                                  batch_size=batch_size, shuffle=False)

#Define the number of classes in your dataset
num_classes = train_generator.num_classes

In [ ]:
from tensorflow.keras.layers import Layer, Attention
import numpy as np

In [ ]:
class LinearAttention(Layer):
    def __init__(self, units):
        super(LinearAttention, self).__init__()
        self.units = units

    def build(self, input_shape):
        self.W_q = self.add_weight(shape=(input_shape[-1], self.units), initializer='uniform', trainable=True)
        self.W_k = self.add_weight(shape=(input_shape[-1], self.units), initializer='uniform', trainable=True)
        self.W_v = self.add_weight(shape=(input_shape[-1], self.units), initializer='uniform', trainable=True)

    def call(self, inputs):
        Q = tf.matmul(inputs, self.W_q)
        K = tf.matmul(inputs, self.W_k)
        V = tf.matmul(inputs, self.W_v)

        attn_scores = tf.matmul(Q, K, transpose_b=True)
        attn_scores = tf.nn.softmax(attn_scores / tf.math.sqrt(tf.cast(self.units, tf.float32)), axis=-1)
        output = tf.matmul(attn_scores, V)

        return output


In [ ]:
# Define your model function with attention
def modelfunction_with_attention(base):
    x = base.output

    # Add Self-Attention Layer
    att_output = LinearAttention(128)(x)

    # Add more layers if needed
    x = tf.keras.layers.GlobalAveragePooling2D()(att_output)
    x = tf.keras.layers.Dropout(0.4)(x)
    predictions = tf.keras.layers.Dense(units=num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.02, l2=0.02))(x)
    model = Model(inputs=base.input, outputs=predictions)
    return model

In [ ]:
def modelfunction(base):
    x = base.output
    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    x = tf.keras.layers.Dropout(0.4)(x)
    predictions = tf.keras.layers.Dense(units=num_classes, activation='softmax', kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.02, l2=0.02))(x)
    model = Model(inputs=base.input, outputs=predictions)
    return model

def get_callbacks(weight):
    checkpoint = ModelCheckpoint(weight, monitor='val_loss', mode='min', save_best_only=True, verbose=1)
    learning_rate_reduction = ReduceLROnPlateau(monitor='val_accuracy', patience=5, verbose=1, factor=0.2, min_lr=0.0002)
    callbacks = [checkpoint, learning_rate_reduction]
    return callbacks

def evaluate(model, generator_test):
    model.evaluate(generator_test)

    y_pred = model.predict(generator_test)
    y_pred_classes = np.argmax(y_pred, axis=1)
    y_true = generator_test.classes
    class_labels = list(generator_test.class_indices.keys())

    print(classification_report(y_true, y_pred_classes))
    cm = confusion_matrix(y_true, y_pred_classes)

    # Plotting the confusion matrix
    plt.figure(figsize=(8, 8))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels)
    plt.show()

def model_training(base, weight, epochs):
    model = modelfunction(base)
    print("\n\n\n-------------------- Model Initialized --------------------")

    callbacks = get_callbacks(weight)
    metrics = ['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall(),
               tfa.metrics.CohenKappa(num_classes=num_classes), tfa.metrics.F1Score(num_classes=num_classes)]
    model.compile(tf.keras.optimizers.Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=metrics)

    history = model.fit(train_generator, steps_per_epoch=366 // batch_size,
                        validation_data=val_generator,  # Add this line
                        epochs=epochs, callbacks=callbacks)
    # Plotting accuracy and loss curves
    plt.figure(figsize=(12, 4))

    # Plot accuracy
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='accuracy')
    plt.plot(history.history['val_accuracy'], label='val_accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.title('Accuracy Over Epochs')
    plt.legend()

    # Plot loss
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.title('Loss Over Epochs')
    plt.legend()

    plt.tight_layout()
    plt.show()

    print("\n\n\n-------------------- Evaluation --------------------")
    evaluate(model, val_generator)

    return model


In [ ]:
# Create and train the model with attention
VGG19 = tf.keras.applications.VGG19(weights='imagenet', include_top=False, input_tensor=None, input_shape=None)
VGG19_model_with_attention = model_training(VGG19, 'VGG19_with_attention.h5', 10)

In [ ]:
import lime
from lime import lime_image
from skimage.segmentation import mark_boundaries
import matplotlib.pyplot as plt
import random


In [ ]:
from lime import lime_image

In [ ]:
from skimage.segmentation import mark_boundaries